In [1]:
import numpy as np
import scipy.sparse
import scipy.stats as stats
import warnings
def solve_adv_diff_eqn(adv_solver='first_order_upwind', a_x=1, Dx=0, Dy=1, dt=0.0005, dx=0.01, dy=0.01, T_end=.5, Lx=5, Ly=5,
                initial_dist=(lambda x,y:  stats.norm.pdf(x, loc=0, scale=1)*stats.norm.pdf(y, loc=0, scale=1))):

    Nt = int(T_end/dt)
    Nx = int(2*Lx/dx)
    Ny = int(2*Ly/dy)
    t = np.linspace(0, T_end, Nt+1)
    x = np.linspace(-Lx, Lx , Nx+1)
    y = np.linspace(-Ly, Ly , Ny+1)


    U = np.zeros((Nx+1, Ny+1), dtype= float)
    U_next = np.zeros((Nx+1, Ny+1), dtype= float)

    Ix = range(0, Nx+1)
    Iy = range(0, Ny+1)
    It = range(0, Nt+1)
    #Initial Conditions
    for i in Ix:
        for j in Iy:
            U[i,j] = initial_dist(x[i], y[j])

    
    mu_x = Dx*dt/(dx)**2
    mu_y = Dy*dt/(dy)**2

    N = (Nx+1)*(Ny+1)
    main   = np.zeros(N)            # diagonal
    sub  = np.zeros(N-1)          # subdiagonal
    sup  = np.zeros(N-1)          # superdiagonal
    lower = np.zeros(N-(Nx+1))     # lower diagonal
    upper = np.zeros(N-(Nx+1))     # upper diagonal
    b      = np.zeros(N)            # right-hand side
    
    m = lambda i,j: j*(Nx+1) + i
    sup[:] = -(0.5*mu_x)
    main[:] = 1 + (mu_x+mu_y)
    sub[:] = -(0.5*mu_x)
    
    lower[:] = - (0.5*mu_y)
    upper[:] = - (0.5*mu_y)
    main[m(0,0):m(Nx+1,0)] = 1 # y left bdy
    main[m(0,j)] = 1 # x lower bdy
    main[m(Nx,j)] = 1 # x upper bdy
    main[m(0,Ny):m(Nx+1,Ny)] = 1 # y right bdy

    diagonals = [lower, sub, main, sup, upper]
    A = scipy.sparse.diags(diagonals,
        offsets=[-(Nx+1), -1, 0, 1, Nx+1],
        shape=(N, N), format='csc')

    def psi_1(theta): return 0.0 
    def psi_2(theta): return float(1/2)
    def psi_3(theta): return  (1/3 + (1/6)*theta)
    def psi_flux(theta): return (np.maximum(0,np.minimum(np.minimum(1, 1/3 + (1/6)*theta), theta)))
    def psi_vl(theta): return (0.5*(theta+np.abs(theta))/(1+np.abs(theta)))

    solvers = {'first_order_upwind': psi_1,
               'second_order_upwind': psi_2,
               'third_order_upwind': psi_3,
               'flux_limited_upwind': psi_flux,
               'van_Leer': psi_vl,
              }

    psi = solvers[str(adv_solver)]
    eps = 10**-25
    
    c = Dx * (dt/dx)
    if abs(c)>1:
        warnings.warn('Method is likely to be unstable, CFL condition failed, c={}>1'.format(c))
    if mu_y > 1./2 or mu_x >1/2:
        warnings.warn(r'Method is likely to be unstable, von Neumann condition failed, \mu ={}>1/2'.format(max(mu_x,mu_y)))
    for n in range(Nt):
        b[m(0,0):m(Nx+1,0)] = 0 #left bdy (j=0)
        b[m(0,Ny):m(Nx+1,Ny)] = 0 # right bdy (j=Ny)
        for j in range(1,Ny): #Is it possible to remove this? Requires m to take vector/slice inputs
            u = U[1:(Nx-1),j] 
            
            u_minus_one_x = np.roll(u,1)
            u_plus_one_x = np.roll(u, -1)
            
            u_minus_one_y = U[1:(Nx-1),j-1]
            u_plus_one_y = U[1:(Nx-1),j+1]
            
            theta_r = (u - u_minus_one_x) / (u_plus_one_x - u +eps)
            theta_r_plus_one = np.roll(theta_r, -1)
            neg_wave_r = (u_plus_one_x + psi(1/(theta_r_plus_one+eps))*(u - u_plus_one_x))
            pos_wave_r = (u +  psi(theta_r)*(u_plus_one_x - u)) 
        
            adv_flux_r = min(a_x,0)*neg_wave_r + max(a_x,0)*pos_wave_r
            diff_flux_r = - 0.5*Dx* (u_plus_one_x - u)/dx
            flux_right = adv_flux_r + diff_flux_r
            flux_left = np.roll(flux_right, 1)
            
            adv_flux_up = 0
            diff_flux_up = - 0.5* Dy* (u_plus_one_y - u)/dx
            flux_up = adv_flux_up + diff_flux_up
            flux_down = np.roll(flux_up, 1)
            
            b[m(0,j)] = 0 # bottom bdy (i=0)
            b[m(Nx,j)] = 0 # top bdy (i=Nx)
            b[m(1,j):m(Nx-1,j)] = u + dt/dx * (flux_left - flux_right) + dt/dy * (flux_down - flux_up)
        c = scipy.sparse.linalg.spsolve(A, b)
        U_next[:,:] = c.reshape(Ny+1,Nx+1).T
        U_next, U = U, U_next    
    return t, U



In [ ]:
#Initial Conditions
def gaussian(x, mean, sd): return stats.norm.pdf(x, loc=mean, scale=sd)
def block(x, centre): return float(x>=centre-0.5 and x<=centre+0.5) #np.array([int(i>=-0.5 and i<=0.5) for i in x])
def bump(x,a):
    f = np.zeros(len(x))
    for i in range(len(x)):
        if a <= np.abs(x[i]) <= a+1:
            f[i] = np.exp(1/(x[i]**2-(a+1)**2))/np.exp(1/(a**2-(a+1)**2))
        elif np.abs(x[i]) < a:
            f[i] = 1
        else:
            f[i] = 0
    return f

In [14]:
%matplotlib qt
x_len = 5
y_len = 5
x_step = 0.02
y_step = 0.02
x_points = int(2*x_len/x_step)
y_points = int(2*y_len/y_step)
T_final = 0.5
x_adv = 0 
x_diff = 2
y_diff = 2
timestep = 0.0001

#IC Parameters:
mean_x = 0
sd_x = 1
mean_y = 0  
sd_y = 1
centre = 0 
initial_data = lambda x,y: gaussian(x,mean_x,sd_x)*gaussian(y, mean_y,sd_y)

t, u = solve_adv_diff_eqn(a_x=x_adv, Dx=x_diff, Dy=y_diff, dt=timestep, dx=x_step, dy=y_step, T_end=T_final,Lx=x_len, Ly=y_len, adv_solver='flux_limited_upwind', initial_dist=initial_data)

X = np.linspace(-x_len, x_len+x_step, x_points+1)
Y = np.linspace(-y_len, y_len + y_step , y_points+1)
X, Y = np.meshgrid(X, Y)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_surface(X,Y,u, cmap=cm.coolwarm, linewidth=0.01)
fig.show()

KeyboardInterrupt: 